In [1]:
from dataclasses import dataclass, asdict, fields
from typing import Optional
import numpy as np
import pandas as pd
import plotly.express as px
from dataPlotter import DataPlotter


## Dataclasses Declaration

In [2]:
@dataclass
class ExecutionEntity:
    Execution_Datetime: np.datetime64
    Endpoint: Optional[str]
    Run_Time: Optional[float]

@dataclass
class Artifact:
    Name: str
    Execution_Datetime: np.datetime64

@dataclass
class ArtifactInfo:
    Artifact_Name: str
    URL: Optional[str]   #url to the object stored on the cloud, can be null
    Artifact_Type: str
    File_Extension: str

@dataclass
class Tests:
    Artifact_Name: str
    Name: str
    Category: str
    Status: str
    Arguments: Optional[str]  # Sometimes details the arguments they receive 
    Execution_Datetime: np.datetime64

@dataclass
class ExecutionTime:
    Execution_name: str  # Can be Test_Name, fixture, or function
    Execution_type: str
    Execution_Datetime: np.datetime64
    Number_Runs: int
    Avg_Time: float
    Min_Time: float
    Total_Time: float

@dataclass
class Failures:
    Artifact_Name: str
    Test_Name: str
    Execution_Datetime: np.datetime64
    Error: str
    Details: Optional[str]  # Retrieved pytest error description

@dataclass
class TestData:
    execution_entity: list[ExecutionEntity]
    artifact: list[Artifact]
    tests: list[Tests]
    execution_time: list[ExecutionTime]
    failures: list[Failures]

def get_field_names(Dataclass: type) -> list[str]:
    return [field.name for field in fields(Dataclass)]


In [3]:
from logExtractor import PytestArtifactLogExtractor
p  = PytestArtifactLogExtractor('pytest_output_cold_storage.20231005T140509.log')

In [4]:
execution_entity, artifact, tests, execution_time, failures = p.log_to_df()

In [5]:
execution_entity_df = pd.DataFrame([asdict(execution_entity)])
artifact_df =pd.DataFrame([asdict(artifact)])
tests_df = pd.DataFrame(list(map(lambda t: asdict(t), tests)))
execution_time_df = pd.DataFrame(asdict(execution_time))
failures_df = pd.DataFrame(asdict(failures))
a=1


In [6]:
tests_df['Status'].value_counts('Status').get('PASSED')

np.float64(0.8)

In [7]:
t = TestData(execution_entity=execution_entity_df, artifact=artifact_df, tests=tests_df, execution_time=execution_time_df, failures=failures_df)

In [8]:
from relatory import PdfMaker
pdf = PdfMaker(t)

pdf.create_pdf()

Status,FAILED,PASSED
Category,,
cold_storage_test.py,1,4


## Schemes

Execution Pipeline: 
    1. Script gera os dados de datetime e total time


    Script > Execution_DateTime, Run_Time
    Run Tests or download artifacts (Github)
    Put generated artifacts into a bucket adressing essential info into its name
    Extract the test, execution time and failures info out of the pytest_artifact


Note:   It might be helpful to create a Run_Id
        the summary does not to have a run id since it will be used mostly for individual runs

__create_df__,__get_time__: classes: ExecutionTime, Tests (Name, Category) #ExecutionTime Name is a FK from Test (Name)

__create_title__: ExecutionEntity

__create_errors_summary__: classes: Failures, Test(Name, Category)

__PdfDataPlotter__: This one is the generator of graphics it will need more than whats expected from the PdfMaker class, so it should be called earlier, generate the graphics as useful images which will be retrieved and added on the pdf

## Dataclasses DFs Declaration

In [9]:
# Create an empty instance (or use default values)
empty_execution_entity = ExecutionEntity(
    Execution_Datetime=np.datetime64("NaT"),  # Not a Time
    Endpoint="",
    Run_Time=0.0
)

# Convert to DataFrame
df_execution_entity = pd.DataFrame([asdict(empty_execution_entity)])
df_execution_entity.dtypes

Execution_Datetime    datetime64[ns]
Endpoint                      object
Run_Time                     float64
dtype: object